In [1]:
import cv2
import numpy as np
from datetime import datetime
import time
from collections import deque

print("✓ Motion-based Activity Tracker initialized!")
print("  Uses: Optical Flow + Background Subtraction")
print("  No models required!")

✓ Motion-based Activity Tracker initialized!
  Uses: Optical Flow + Background Subtraction
  No models required!


In [2]:
def calculate_angle(a, b, c):
    """Calculate angle between three points"""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

def detect_motion_regions(frame, bg_subtractor):
    """Detect motion regions using background subtraction"""
    # Apply background subtractor
    fg_mask = bg_subtractor.apply(frame)
    
    # Denoise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours, fg_mask

def get_optical_flow(prev_gray, curr_gray):
    """Calculate dense optical flow"""
    flow = cv2.calcOpticalFlowFarneback(
        prev_gray, curr_gray, None, 
        pyr_scale=0.5, levels=3, winsize=15,
        iterations=3, poly_n=5, poly_sigma=1.2, flags=0
    )
    return flow

def calculate_motion_intensity(flow):
    """Calculate overall motion intensity from optical flow"""
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    return np.mean(magnitude)

def draw_motion_vectors(frame, flow, step=16):
    """Draw motion vectors on frame"""
    h, w = frame.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    
    # Draw vectors
    for i in range(len(x)):
        if abs(fx[i]) > 1 or abs(fy[i]) > 1:  # Only draw significant motion
            cv2.arrowedLine(frame, (x[i], y[i]), 
                          (int(x[i] + fx[i]*3), int(y[i] + fy[i]*3)),
                          (0, 255, 0), 1, tipLength=0.3)
    return frame

In [3]:
def draw_motion_regions(frame, contours):
    """Draw bounding boxes around motion regions"""
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter small movements
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)
            
            # Calculate center
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
    
    return frame

def create_heatmap(motion_history):
    """Create motion heatmap from history"""
    if len(motion_history) == 0:
        return None
    
    # Stack all motion masks
    heatmap = np.sum(motion_history, axis=0).astype(np.float32)
    heatmap = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX)
    heatmap = heatmap.astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    return heatmap

In [4]:
class ActivityCounter:
    """Count activities based on motion patterns"""
    def __init__(self, activity_type="jumping"):
        self.activity_type = activity_type
        self.counter = 0
        self.stage = None
        self.motion_history = deque(maxlen=30)
        self.threshold_high = 15
        self.threshold_low = 5
    
    def update(self, motion_intensity):
        """Update counter based on motion intensity"""
        self.motion_history.append(motion_intensity)
        
        if len(self.motion_history) < 10:
            return motion_intensity, "warming up"
        
        # Detect activity cycles based on motion intensity
        avg_motion = np.mean(list(self.motion_history)[-10:])
        
        if avg_motion > self.threshold_high:
            stage = "active"
        elif avg_motion < self.threshold_low:
            stage = "rest"
        else:
            stage = self.stage
        
        # Count transitions from active to rest
        if self.stage == "active" and stage == "rest":
            self.counter += 1
        
        self.stage = stage
        return motion_intensity, stage
    
    def reset(self):
        """Reset counter"""
        self.counter = 0
        self.stage = None
        self.motion_history.clear()

In [5]:
def motion_tracker(camera_id=0, show_flow=True, show_heatmap=True, count_activity=False):
    """
    Real-time motion tracking using OpenCV (no ML models)
    
    Args:
        camera_id: Camera index (default: 0)
        show_flow: Show optical flow vectors
        show_heatmap: Show motion heatmap
        count_activity: Enable activity counting
    
    Controls:
        Q - Quit
        R - Reset counter
        S - Save frame
        F - Toggle flow vectors
        H - Toggle heatmap
    """
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print("❌ Error: Could not open camera")
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    # Initialize background subtractor
    bg_subtractor = cv2.createBackgroundSubtractorMOG2(
        history=500, varThreshold=16, detectShadows=False
    )
    
    # Activity counter
    counter = ActivityCounter() if count_activity else None
    
    prev_gray = None
    prev_time = 0
    frame_count = 0
    motion_history = deque(maxlen=30)
    
    print("🎯 Starting Motion Tracker... Press Q to quit")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Calculate FPS
        curr_time = time.time()
        fps = 1 / (curr_time - prev_time) if prev_time > 0 else 0
        prev_time = curr_time
        frame_count += 1
        
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect motion regions
        contours, fg_mask = detect_motion_regions(frame, bg_subtractor)
        motion_history.append(fg_mask)
        
        # Draw motion regions
        frame = draw_motion_regions(frame, contours)
        
        # Optical flow
        if prev_gray is not None:
            flow = get_optical_flow(prev_gray, gray)
            motion_intensity = calculate_motion_intensity(flow)
            
            if show_flow:
                frame = draw_motion_vectors(frame, flow)
            
            # Activity counting
            if counter:
                intensity, stage = counter.update(motion_intensity)
                h, w = frame.shape[:2]
                
                # Activity panel
                overlay = frame.copy()
                cv2.rectangle(overlay, (w - 300, 10), (w - 10, 240), (0, 0, 0), -1)
                cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
                
                cv2.putText(frame, 'ACTIVITY', (w - 280, 50),
                           cv2.FONT_HERSHEY_BOLD, 1.0, (0, 255, 255), 2)
                cv2.putText(frame, 'COUNT', (w - 280, 90),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                cv2.putText(frame, str(counter.counter), (w - 280, 140),
                           cv2.FONT_HERSHEY_BOLD, 2, (0, 255, 0), 4)
                
                stage_color = (0, 255, 0) if stage == "active" else (0, 165, 255)
                cv2.putText(frame, f'Stage: {stage}', (w - 280, 180),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, stage_color, 2)
                cv2.putText(frame, f'Motion: {intensity:.1f}', (w - 280, 215),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        
        prev_gray = gray.copy()
        
        # FPS display
        cv2.putText(frame, f'FPS: {fps:.1f}', (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, f'Frame: {frame_count}', (20, 70),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        cv2.putText(frame, f'Regions: {len(contours)}', (20, 100),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        # Controls hint
        cv2.putText(frame, "Q:Quit | R:Reset | S:Save | F:Flow | H:Heatmap", 
                   (10, frame.shape[0] - 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Show heatmap
        if show_heatmap and len(motion_history) > 0:
            heatmap = create_heatmap(list(motion_history))
            if heatmap is not None:
                # Resize heatmap to fit corner
                h_small = cv2.resize(heatmap, (320, 240))
                frame[10:250, 10:330] = cv2.addWeighted(
                    frame[10:250, 10:330], 0.5, h_small, 0.5, 0
                )
        
        cv2.imshow('Motion Tracker', frame)
        
        # Handle keys
        key = cv2.waitKey(1) & 0xFF
        if key in (ord('q'), ord('Q')):
            break
        elif key in (ord('r'), ord('R')) and counter:
            counter.reset()
            print("✓ Counter reset!")
        elif key in (ord('s'), ord('S')):
            img_name = f'frame_{datetime.now().strftime("%Y%m%d_%H%M%S")}.jpg'
            cv2.imwrite(img_name, frame)
            print(f"✓ Saved: {img_name}")
        elif key in (ord('f'), ord('F')):
            show_flow = not show_flow
            print(f"✓ Flow vectors: {'ON' if show_flow else 'OFF'}")
        elif key in (ord('h'), ord('H')):
            show_heatmap = not show_heatmap
            print(f"✓ Heatmap: {'ON' if show_heatmap else 'OFF'}")
    
    cap.release()
    cv2.destroyAllWindows()
    
    if counter:
        print(f"✅ Done! Total activities: {counter.counter}")
    else:
        print(f"✅ Done! Tracked {frame_count} frames.")

In [6]:
# Basic motion tracking
motion_tracker(camera_id=0, show_flow=True, show_heatmap=True)

# With activity counting:
# motion_tracker(camera_id=0, count_activity=True)

# Just motion regions (no flow/heatmap):
# motion_tracker(camera_id=0, show_flow=False, show_heatmap=False)

🎯 Starting Motion Tracker... Press Q to quit
✅ Done! Tracked 200 frames.
